# Streaming Patterns - Advanced

Production-ready streaming patterns with async, cancellation, and UI enhancements.

In [ ]:
from openai import OpenAI, AsyncOpenAI
from dotenv import load_dotenv
import asyncio
import time
import tiktoken

load_dotenv()
client = OpenAI()
async_client = AsyncOpenAI()

## Pattern 1: Async Streaming

Stream responses asynchronously for better concurrency.

In [ ]:
async def async_stream_chat(message: str) -> str:
    """
    Stream chat response asynchronously.
    """
    stream = await async_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": message}],
        stream=True
    )
    
    full_response = ""
    
    async for chunk in stream:
        delta = chunk.choices[0].delta.content
        if delta:
            full_response += delta
            print(delta, end='', flush=True)
    
    print()  # Newline
    return full_response

# Test async streaming
response = await async_stream_chat("Tell me a fun fact about space.")
print(f"\n✅ Completed: {len(response)} characters")

## Pattern 2: Multiple Concurrent Streams

Stream multiple responses at the same time!

In [ ]:
async def labeled_stream(label: str, message: str) -> tuple:
    """
    Stream with label prefix for concurrent streams.
    """
    stream = await async_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": message}],
        stream=True,
        max_tokens=50  # Keep short for demo
    )
    
    full_response = ""
    
    async for chunk in stream:
        delta = chunk.choices[0].delta.content
        if delta:
            full_response += delta
            print(f"[{label}] {delta}", end='', flush=True)
    
    print(f"\n[{label}] ✓ Done")
    return label, full_response

# Stream 3 responses concurrently
async def concurrent_streams():
    tasks = [
        labeled_stream("A", "Fun fact about Python?"),
        labeled_stream("B", "Fun fact about JavaScript?"),
        labeled_stream("C", "Fun fact about Rust?")
    ]
    
    results = await asyncio.gather(*tasks)
    return results

print("Streaming 3 responses concurrently:\n")
results = await concurrent_streams()
print(f"\n✅ All {len(results)} streams complete!")

## Pattern 3: Progress Indicators

Show progress while streaming.

In [ ]:
import sys

def stream_with_progress(message: str):
    """
    Stream with word count progress indicator.
    """
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": message}],
        stream=True
    )
    
    full_content = ""
    word_count = 0
    
    print("Response: ", end='', flush=True)
    
    for chunk in stream:
        delta = chunk.choices[0].delta.content
        
        if delta:
            full_content += delta
            
            # Count words in delta
            word_count += len(delta.split())
            
            # Display content
            print(delta, end='', flush=True)
    
    print(f"\n\n📊 Progress: {word_count} words, {len(full_content)} characters")
    return full_content

result = stream_with_progress("Write a paragraph about AI.")

## Pattern 4: Real-Time Token Counter

Count tokens as they stream in.

In [ ]:
def stream_with_live_token_count(message: str, model="gpt-3.5-turbo"):
    """
    Stream with real-time token counting.
    """
    encoding = tiktoken.encoding_for_model(model)
    
    stream = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": message}],
        stream=True
    )
    
    full_content = ""
    token_count = 0
    update_interval = 5  # Update count every N tokens
    
    print("Streaming... ", end='', flush=True)
    
    for chunk in stream:
        delta = chunk.choices[0].delta.content
        
        if delta:
            full_content += delta
            print(delta, end='', flush=True)
            
            # Update token count periodically
            new_token_count = len(encoding.encode(full_content))
            if new_token_count >= token_count + update_interval:
                token_count = new_token_count
                # Print token count inline
                print(f" [{token_count}t]", end='', flush=True)
    
    final_tokens = len(encoding.encode(full_content))
    print(f"\n\n📊 Final token count: {final_tokens}")
    
    return full_content, final_tokens

content, tokens = stream_with_live_token_count(
    "Explain how neural networks learn."
)

## Pattern 5: Cancellation Support

Allow users to stop streaming mid-response.

In [ ]:
import threading

class CancellableStream:
    """
    Stream that can be cancelled.
    """
    def __init__(self):
        self.cancelled = False
    
    def cancel(self):
        """Cancel the stream."""
        self.cancelled = True
        print("\n\n⚠️  Stream cancelled by user!")
    
    def stream(self, message: str, model="gpt-3.5-turbo"):
        """Stream with cancellation support."""
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": message}],
            stream=True
        )
        
        full_content = ""
        
        print("Streaming (will auto-cancel after 3 seconds for demo)...\n")
        
        for chunk in stream:
            if self.cancelled:
                break
            
            delta = chunk.choices[0].delta.content
            if delta:
                full_content += delta
                print(delta, end='', flush=True)
        
        if not self.cancelled:
            print("\n\n✅ Stream completed naturally")
        
        return full_content

# Demo: Auto-cancel after 3 seconds
cancellable = CancellableStream()

# Set up auto-cancel for demo
timer = threading.Timer(3.0, cancellable.cancel)
timer.start()

result = cancellable.stream("Write a long essay about space exploration.")
timer.cancel()  # Clean up timer

print(f"\nReceived {len(result)} characters before cancellation")

## Pattern 6: Buffered Streaming

Buffer chunks for smoother display.

In [ ]:
import time

def buffered_stream(message: str, buffer_size: int = 5):
    """
    Stream with buffering for smoother display.
    
    Args:
        message: User message
        buffer_size: Number of tokens to buffer before displaying
    """
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": message}],
        stream=True
    )
    
    buffer = ""
    full_content = ""
    
    print("Buffered streaming:\n")
    
    for chunk in stream:
        delta = chunk.choices[0].delta.content
        
        if delta:
            buffer += delta
            
            # Flush buffer when it reaches size
            if len(buffer.split()) >= buffer_size:
                print(buffer, end='', flush=True)
                full_content += buffer
                buffer = ""
                time.sleep(0.1)  # Simulate smooth display
    
    # Flush remaining buffer
    if buffer:
        print(buffer, end='', flush=True)
        full_content += buffer
    
    print("\n\n(Notice smoother 'chunked' display)")
    return full_content

result = buffered_stream("Describe machine learning in simple terms.")

## Pattern 7: Conversation Streaming

Stream within a conversation manager.

In [ ]:
class StreamingConversation:
    """
    Conversation manager with streaming support.
    """
    def __init__(self, system_prompt: str = "You are a helpful assistant."):
        self.messages = [{"role": "system", "content": system_prompt}]
        self.client = OpenAI()
        self.encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")
    
    def stream_reply(self, user_message: str) -> dict:
        """Send message and stream response."""
        # Add user message
        self.messages.append({"role": "user", "content": user_message})
        
        # Stream response
        stream = self.client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=self.messages,
            stream=True
        )
        
        full_response = ""
        
        print("Assistant: ", end='', flush=True)
        
        for chunk in stream:
            delta = chunk.choices[0].delta.content
            if delta:
                full_response += delta
                print(delta, end='', flush=True)
        
        print()  # Newline
        
        # Add assistant response to history
        self.messages.append({"role": "assistant", "content": full_response})
        
        return {
            'content': full_response,
            'tokens': len(self.encoding.encode(full_response)),
            'turn': len([m for m in self.messages if m['role'] == 'user'])
        }

# Test conversation streaming
conv = StreamingConversation()

print("User: What is Python?")
result1 = conv.stream_reply("What is Python?")

print(f"\n[{result1['tokens']} tokens]\n")

print("User: What can I build with it?")
result2 = conv.stream_reply("What can I build with it?")

print(f"\n[{result2['tokens']} tokens]")
print(f"\n✅ Conversation has {result2['turn']} turns")

## Production Tips

### Performance
- Use async streaming for handling multiple users
- Buffer tokens for smoother UI updates
- Don't update UI on every single token (rate limit updates)

### UX
- Always show streaming progress
- Provide cancel button for long responses
- Show "typing" indicator before first token
- Display token/cost estimates

### Error Handling
- Stream can fail mid-response
- Always have fallback for network errors
- Save partial responses on error
- Implement retry logic

### Token Management
- Count tokens manually (not in stream)
- Set max_tokens to prevent runaway costs
- Track streaming costs same as non-streaming

## Practice Exercises

1. Build async streaming chat that handles 10 concurrent users
2. Add progress bar that fills as response streams
3. Implement "Stop" button that cancels stream
4. Create buffered display with typing animation
5. Build streaming conversation with save/load support